In [12]:
import numpy as np, crocoddyl, random; crocoddyl.switchToNumpyArray()
import pandas as pd, xarray as xr

data = []
for _ in range(100000):
    initial_config = [random.uniform(-1.99, 1.99), random.uniform(-1.99, 1.99), 0]            
    model = crocoddyl.ActionModelUnicycle()

    model.costWeights = np.matrix([1, 0.3]).T

    problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve([], [], 1000)
    position = []
    # Attach x, y, theta
    position.extend(i for i in ddp.xs[0])
    # Attach linear_velocity, angular_velocty
    position.extend(i for i in ddp.us[0])
    # Attach cost
    position.append(sum(d.cost for d in ddp.datas()))
    # Attach the number of iterations
    position.append(ddp.iter)
    data.append(position)
    
data = np.array(data)
df = pd.DataFrame(data[0:,0:], columns = ["x_position", "y_position", "z_position", "linear_velocity", "angular_velocity", "value_function", "iterations"])
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head(1)

(100000, 7)


,x_position,y_position,z_position,linear_velocity,angular_velocity,value_function,iterations
0,-1.869853,-0.139411,0.0,5.236817,0.569089,6.293099,6.0


In [1]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
plt.scatter(data[:, 0], data[:, 1], c=data[:, 5], cmap = 'jet', alpha = 0.8, linewidths = 0)
plt.axis("image")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.colorbar()
plt.show()

NameError: name 'data' is not defined

In [ ]:
import plotly.graph_objects as go
import numpy as np

import plotly.express as px
fig = px.scatter(df, x="x_position", y="y_position", color="value_function",
                 hover_data=['value_function'],color_continuous_scale=px.colors.diverging.Tealrose, width=800, height=600, color_continuous_midpoint=2,
                render_mode="svg")

    
fig.show()